## 什么是模型堆叠？
堆叠是一个接一个使用不同机器学习模型的过程，您可以从每个模型中添加预测，从而形成新特性。

通常有两个不同的堆叠变体：变体 A 和  B。在本文中，我将重点介绍变体 A，与变体 B 相比，变体 A 似乎能获得更好的结果，因为模型更容易与变体 B 中的训练数据过拟合。这可能也是许多从业人员使用变体 A 的原因，尽管它不能消除过拟合。

注意，没有正确的实现模型堆叠的方法 (David H. Wolpert)，因为模型堆叠仅描述将多个模型与最终的广义模型相结合的过程。变体 A  在实践中被证明效果很好，因此我选择使用它。

## 为什么要使用堆叠？
 在本文中我提到过更好的结果。现在，让我来证明在使用模型堆叠时如何以及为何可以获得更好的结果。

竞争的赢家者和从业者选择使用模型堆叠，它很简单。您的元学习器的归纳能力要优于单个模型。也就是说，与仅使用单个模型相比，元学习器可以对看不见的数据做出更好的预测。
关于该主题的一篇很好的论文：Stacked Generalization，作者 David H. Wolpert。Wolpert  认为，模型堆叠可以推断出特定数据集上模型的偏差，以便我们稍后可以在元学习器中校正偏差。
在模型堆叠变得流行且广泛使用之前，存在一种称为贝叶斯模型平均 (BMA) 的技术。在此模型中，权重是模型的后验概率。但是，Bertrand Clarke 的实验证据表明，具有交叉验证功能的模型堆叠可以执行  BMA，同时具备健全性，因为模型堆叠对数据集中的变化不那么敏感。
我认为您最应该注意的论点是第一点。大多数人使用模型堆叠来获得更好的结果。但如果您想知道它为什么有效，我建议您阅读 Wolpert 的论文。

## 分步介绍：用于在 Python中实现堆叠的代码

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import warnings

In [2]:
def dataset_to_df(load):
    # Load the input data into the dataframe
    df = pd.DataFrame(load.data, columns=load.feature_names)
 
    # Add the output data into the dataframe
    df['label'] = pd.Series(load.target)
 
    # Return the dataframe
    return df
 
df = dataset_to_df(load_boston())

In [3]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,label
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1-13 是我们的输入，而 14 是我们在给定看不见的测试数据集的情况下尝试尽可能精确地预测的输出。

    1. CRIM：城镇人均犯罪率
    2. ZN：划定面积超过 25,000 平方英尺的住宅用地比例
    3.INDUS：每个城镇非零售业务用地的比例
    4.CHAS：Charles River 哑变量（如果土地限定河流，那么值为 1；否则值为 0）
    5. NOX：一氧化氮浓度（千万分之一）
    6.RM：每个住宅的平均房间数
    7.AGE：1940 年之前建造的自有住房的比例
    8.DIS：到波士顿的五个就业中心的加权距离
    9.RAD：辐射式公路的可达性指数
    10.TAX：每 10,000 美元的全值财产税率
    11.PTRATIO：城镇师生比率
    12.B：1000(Bk-0.63)^2，其中 Bk 是城镇黑人比例
    13.LSTAT：地位较低人口百分比 (%)
    14.MEDV (label)：自有住房的中位数价值（以 1000 美元计）

## 堆叠前网格搜索

In [13]:
from sklearn.model_selection import train_test_split
 
# Getting the output variable
y = df['label']
 
# Getting the input variables
X = df.drop(['label'], axis=1)
 
# Diving our input and output into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y,
                                    test_size=0.33,
                                    random_state=42
                                   )

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
 
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data,
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       scoring_test=r2_score, do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv,
        # n_jobs=-1,
        n_jobs=1,
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    best_model = fitted_model.best_estimator_
 
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
 
    score = scoring_test(y_test_data, pred)
 
    return [best_model, pred, score]

In [16]:
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
 


In [17]:
# Defining our estimator, the algorithm to optimize
models_to_train = [XGBRegressor(), LGBMRegressor(), RandomForestRegressor()]
 
# Defining the hyperparameters to optimize
grid_parameters = [
    { # XGBoost
        'n_estimators': [400, 700, 1000],
        'colsample_bytree': [0.7, 0.8],
        'max_depth': [3, 5, 10, 13],
        # 'reg_alpha': [1.1, 1.2, 1.3],
        # 'reg_lambda': [1.1, 1.2, 1.3],
        'subsample': [0.7, 0.8, 0.9]
    },
    { # LightGBM
        'n_estimators': [400, 700, 1000],
        'learning_rate': [0.12],
        'colsample_bytree': [0.7, 0.8],
        'max_depth': [3, 4,5,6,7],
        'num_leaves': [10, 20]
        # 'reg_alpha': [1.1, 1.2],
        # 'reg_lambda': [1.1, 1.2],
        # 'min_split_gain': [0.3, 0.4],
        # 'subsample': [0.8, 0.9],
        # 'subsample_freq': [10, 20]
    },
    { # Random Forest
        'max_depth':[3, 5, 7,10],
        'n_estimators':[100, 200, 400, 600, 900]
        # 'max_features':[2, 4, 6, 8, 10]
    }
]

In [18]:
models_preds_scores = []
 
for i, model in enumerate(models_to_train):
    params = grid_parameters[i]
 
    result = algorithm_pipeline(X_train, X_test, y_train, y_test,
                                 model, params, cv=5)
    models_preds_scores.append(result)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7, total=   0.3s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7 
[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7 
[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7 
[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.7, total=   0.2s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.8 
[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.8, total=   0.2s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.8 
[CV]  colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.8, total=   0.3s
[CV] colsample_bytree=0.7, max_depth=3, n_estimators=400, subsample=0.8 
[CV]  colsample_bytree=0.7, max_de

...

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   42.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................... max_depth=3, n_estimators=100, total=   0.2s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .................... max_depth=3, n_estimators=100, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] .................... max_depth=3, n_estimators=100, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .................... max_depth=3, n_estimators=100, total=   0.1s
[CV] max_depth=3, n_estimators=100 ...................................
[CV] .................... max_depth=3, n_estimators=100, total=   0.1s
[CV] max_depth=3, n_estimators=200 ...................................
[CV] .................... max_depth=3, n_estimators=200, total=   0.3s
[CV] max_depth=3, n_estimators=200 ...................................
[CV] .................... max_depth=3, n_estimators=200, total=   0.3s
[CV] max_depth=3, n_estimators=200 ...................................
[CV] .................... max_depth=3, n_estimators=200, total=   0.3s
[CV] max_depth=3, n_estimators=200 ...................................
[CV] .................... max_depth=3, n_estimators=200, total=   0.3s
[CV] max_depth=3, n_estimators=200 ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.2min finished


In [19]:
for result in models_preds_scores:
    print('Model: {0}, Score: {1}'.format(type(result[0]).__name__, result[2]))

Model: XGBRegressor, Score: 0.8670969934251533
Model: LGBMRegressor, Score: 0.8652702136410685
Model: RandomForestRegressor, Score: 0.8659754461294342


## 堆叠模型

在对现有包进行一些研究后，我发现了 pystacknet 和 mlxtend。在本文中，我使用 mlxtend 是因为它与 scikit-learn 很好地集成，后者定义了XGBoost、LightGBM 和其他包遵循的出色 API。

我们首先从运行此代码所需的包中导入类。我们假设没有用于堆叠的超参数。这意味着我们为每个模型使用预定义的超参数。

In [42]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
 
xgb = XGBRegressor(seed=1)
lgb = LGBMRegressor(seed=1)
rf = RandomForestRegressor()
ridge = Ridge()
lasso = Lasso()
svr = SVR(kernel='linear')

In [45]:
stack = StackingCVRegressor(regressors=(ridge, lasso, svr, rf, lgb, xgb),
                            meta_regressor=lgb, cv=12,
                            # meta_regressor=xgb, cv=10,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=False,
                            random_state=42)
 
stack.fit(X_train, y_train)
 
X_test.columns = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12']
pred = stack.predict(X_test)
score = r2_score(y_test, pred)

In [46]:
score

0.8995248440162951

## 机器学习中的集成方法（4）--Stacking（堆叠法）

Stacking 就是当用初始训练数据学习出若干个基学习器后，将这几个学习器的预测结果作为新的训练集，来学习一个新的学习器。Stacking 的基础层通常包括不同的学习算法，因此stacking ensemble往往是异构的。

作者：Byte猫
链接：https://www.jianshu.com/p/7fc9aa03ec11
来源：简书
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

In [48]:
#-*- coding:utf-8 -*-
'''
Stacking方法
'''
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings(action='ignore')

#========================================================
#  载入iris数据集
#========================================================

iris = load_iris()
X = iris.data[:,:5]
y = iris.target

print('feature=',X)
print('target=',y)

#========================================================
#  实现Stacking集成
#========================================================

def StackingMethod(X, y):
    '''
    Stacking方法实现分类
    INPUT -> 特征, 分类标签
    '''
    scaler = StandardScaler() # 标准化转换
    scaler.fit(X)  # 训练标准化对象
    traffic_feature= scaler.transform(X)   # 转换数据集
    feature_train, feature_test, target_train, target_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=0)

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(random_state=1)
    clf3 = GaussianNB()

    sclf = StackingClassifier(classifiers=[clf1, clf2, clf3],
                              # use_probas=True, 类别概率值作为meta-classfier的输入
                              # average_probas=False,  是否对每一个类别产生的概率值做平均
                            #   meta_classifier=LogisticRegression())
                              use_features_in_secondary=True,
                              meta_classifier=RandomForestClassifier())

    sclf.fit(feature_train, target_train)

    # 模型测试
    predict_results = sclf.predict(feature_test)
    print(accuracy_score(predict_results, target_test))
    conf_mat = confusion_matrix(target_test, predict_results)
    print(conf_mat)
    print(classification_report(target_test, predict_results))

    # 5折交叉验证
    for clf, label in zip([clf1, clf2, clf3, sclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'StackingModel']):
        scores = model_selection.cross_val_score(clf, X, y, cv=5, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

    return sclf

#========================================================
#  主程序
#========================================================

if __name__ == '__main__':

    model = StackingMethod(X, y)

feature= [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6

另外一种方法是对训练集中的特征维度进行操作的，这次不是给每一个基分类器全部的特征，而是给不同的基分类器分配不同的特征，即比如基分类器1训练前半部分特征，基分类器2训练后半部分特征（可以通过sklearn 的pipelines 实现）。最终通过StackingClassifier组合起来。

In [41]:
#-*- coding:utf-8 -*-
'''
Stacking方法
'''
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import ColumnSelector
from mlxtend.classifier import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings(action='ignore')

#========================================================
#  载入iris数据集
#========================================================

iris = load_iris()
X = iris.data[:,:5]
y = iris.target

print('feature=',X)
print('target=',y)

#========================================================
#  实现Stacking集成
#========================================================

def StackingMethod(X, y):
    '''
    Stacking方法实现分类
    INPUT -> 特征, 分类标签
    '''
    scaler = StandardScaler() # 标准化转换
    scaler.fit(X)  # 训练标准化对象
    traffic_feature= scaler.transform(X)   # 转换数据集
    feature_train, feature_test, target_train, target_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=0)

    pipe1 = make_pipeline(ColumnSelector(cols=(0, 1,2)),  
                          LogisticRegression())
    pipe2 = make_pipeline(ColumnSelector(cols=(2, 3)),  
                          LogisticRegression())

    sclf = StackingClassifier(classifiers=[pipe1, pipe2],
                              meta_classifier=LogisticRegression())

    sclf.fit(feature_train, target_train)

    # 模型测试
    predict_results = sclf.predict(feature_test)
    print(accuracy_score(predict_results, target_test))
    conf_mat = confusion_matrix(target_test, predict_results)
    print(conf_mat)
    print(classification_report(target_test, predict_results))

    return sclf

#========================================================
#  主程序
#========================================================

if __name__ == '__main__':

    model = StackingMethod(X, y)

feature= [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6

## 使用源码实现Stacking方法

In [ ]:
# coding=utf-8
import imp
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
x = iris.data
y = iris.target

import pandas as pd

x = pd.DataFrame(x, columns=['s_len','s_wid','p_len','p_wid'])

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.svm import SVC

class StackingAveragedModels(BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):

        # 训练第一级学习器
        layer1_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            model.fit(X, y)
            y_pred = model.predict(X)
            layer1_predictions[:, i] = y_pred

        # 训练第二级学习器
        self.meta_model.fit(layer1_predictions, y)
        return self

    def predict(self, X):
        temp = np.column_stack([model.predict(X) for model in self.base_models])
        return self.meta_model.predict(temp)

clf = StackingAveragedModels(base_models=[SVC(gamma='scale'),
                                          SVC(gamma='scale')],
                             meta_model=SVC(gamma='scale'))

pipe = Pipeline(steps=[
              ('Clf', clf)
              ])

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.5, random_state= 666)
pipe.fit(x_train, y_train)

print('Test accuracy is %.3f' % pipe.score(x_test, y_test))

In [1]:
from sklearn.metrics import accuracy_score
y_pred = [0, 2, 1, 3]
y_true = [0, 1, 2, 3]
accuracy_score(y_true, y_pred)

0.5